<a href="https://colab.research.google.com/github/djgranizo/cx4240_Stars/blob/master/GMMStars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture as GMM
from sklearn.decomposition import PCA

In [0]:
# Accessing Data

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

def isFloat(string):
    try:
        float(string)
        return True
    except ValueError:
        return False

# Open our new sheet and read some data.
worksheet = gc.open('Numeric Hypatia Data').sheet1
rows = worksheet.get_all_values()
headers = rows[0]
star_data = np.array([[float(x) if isFloat(x) else np.nan for x in row ] for row in rows[1:]])
#star_data
numstars = len(star_data)

col_mean = np.nanmean(star_data, axis=0)
inds = np.where(np.isnan(star_data))
star_data[inds] = np.take(col_mean, inds[1])
#star_data

#star_data[:,15]

In [0]:
"""Evaluate GMM at each star and assign each star to the 
cluster that it has the highest probability of belonging to""" 

gmm = GMM(n_components=4) #4 clusters: F, G, K, and M
gmm.fit(star_data)

clusters_means = gmm.means_
cluster_weights_ = gmm.weights_
cluster_covariance = gmm.covariances_

clusterAssignment = gmm.fit_predict(star_data)
probabilities = gmm.predict_proba(star_data)

In [0]:
#this block creates <hostprobability> array, with length numclusters, which calculates the probability of planet host in each cluster
clusterAssignments=clusterAssignment
numclusters=4
hostprobability=[]
hostdata=star_data[:,-1]
starspercluster=np.zeros(numclusters)
for c in range(numclusters):
  for assignment in clusterAssignment:
    if(assignment==c):
      starspercluster[c]=starspercluster[c]+1
for c in range(numclusters):
  numstars=starspercluster[c]
  numhosts=0
  for a in range(len(clusterAssignments)):
    assignment=clusterAssignment[a]
    if assignment==c:
      numstars=numstars+1
      if (hostdata[a]==1):
        numhosts=numhosts+1
  hostprobability.append(numhosts/numstars)
print(hostprobability)

[0.07520325203252033, 0.007097791798107256, 0.2, 0.0]


In [0]:
#this block creates purity test to compare our cluster assignments to spectral class
spectraltype=star_data[:,15] #this the the spectral class assignment for each star--this needs to be imported from the spreadsheet--should be a number 1-4
clusterassignments=clusterAssignment
purities=[] #purity for each cluster
clusters=[i for i in range(numclusters)]
print("spectral types: "+str(spectraltype))
for c in range(numclusters): #for each cluster, evaluate purity
  starsincluster=[]
  for i in range(len(clusterassignments)):
    assignment=clusterassignments[i] #cluster assignment of star i
    if assignment==c:
      starsincluster.append(i)
  numspectraltype=np.ones(40) #calculate the number of stars of each spectral type, save it to this array--this is an array that looks like [500,251,30,65]
  #, where this cluster has 500 stars that are spectral type 1
  for cc in range(len(clusters)):
    for s in range(len(starsincluster)):
      currspectraltype=spectraltype[s] #this is a number 1-30
      numspectraltype[int(currspectraltype)]=numspectraltype[int(currspectraltype)]+1 #increment the number of stars of the spectral type that was found here
  numspectraltype=numspectraltype/len(starsincluster) #normalize by total # stars in this cluster to get the percent probabilities-this is an array that looks like 
  #[0.59, 0.29, 0.03, 0.07], where 0.59 of the stars in this cluster are spectral type 1
  maxindex=0
  maxval=0
  for i in range(len(numspectraltype)):
    if(numspectraltype[i] > maxval):
      maxval=numspectraltype[i]
      maxindex=i
  purities.append(maxval) # so the purity of the above example would be 0.59
print("purities: "+str(purities))

spectral types: [11. 10. 10. ... 25. 35. 10.]
purities: [0.491869918699187, 0.48738170347003157, 0.47215189873417723, 0.45586268390548373]


In [0]:
'''
def centerX(star_data):
  return star_data-np.mean(star_data, axis = 0)
    
def svd(X):
  U, S, V = np.linalg.svd(X)
  return U, S, V
  
def rebuildX(U, S, V, k):
  S = np.diag(S)[:k,:k]
  print(U.shape)
  U = U[:, :k]
  UkSk = np.matmul(U, S)
  V = V[:, :k]
  UkSkVk = np.matmul(UkSk, V)
  
  #return np.matmul(     np.matmul(U[:, [i for i in range(k)]], np.diag(S)[:k, :k])            , V[0:k])

X = centerX(star_data)
U, S, V = svd(X)
print(U.shape)
print(V.shape)
print(S.shape)
k = 2
dim_red = rebuildX(U, S, V, 2)
#print(dim_red)

print(dim_red.shape)
'''

sklearn.decomposition.PCA(n_components=2, copy=True, whiten=False, svd_solver=’auto’, tol=0.0, iterated_power=’auto’, random_state=None)

(6156, 6156)
(24, 24)
(24,)
(6156, 6156)


ValueError: ignored